In [1]:
# OpenAI API Key 정의
# **주의!** OpenAI API Key 는 외부에 노출하지 않도록 주의한다. 
# 4월 17일 이후에는 이전에 사용하던 사용자 키(User Key)보다 프로젝트별로 구분할 수 있는 프로젝트 키(Project Key)를 활용하는 것을 권장
# 일반적으로 시스템의 환경변수에 숨겨놓고 소스코드에는 노출하지 않지만, 
# 해당 과정은 파이썬 활용에 관한 내용이므로 예제에서는 생략한다.
# [모델 문서](https://platform.openai.com/docs/models)에는 사용가능한 모델의 이름이 나열되어있고, OpenAI에서 모델을 업데이트할 때마다 이곳에 명시하므로 자주 확인하면 좋다.

try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print(f"> 구글 코랩의 보안 비밀에서 OPENAI_API_KEY를 불러옵니다.")

except:
    OPENAI_API_KEY = "YOUR_OPENAI_PROJECT_KEY"
    print(f"> 로컬 환경에서 설정한 OPENAI_API_KEY를 사용합니다.")

OPENAI_MODEL = "gpt-4-turbo"

In [2]:
# OpenAI API 패키지 import
try:
    import openai

except ModuleNotFoundError:
    # OpenAI API 패키지 설치
    !pip install -r requirements.txt

finally:
    import openai

if "1.23.1" != openai.VERSION:
    raise "예제는 2024-04-17 update 버전을 기준으로 작성하였습니다."

### [Assistants Tool] Function Calling

어시스턴트가 OpenAI 외부에서 호스팅하는 함수를 호출하여 그 결과를 제공

In [21]:
import datetime
import yfinance as yf
import nest_asyncio
import asyncio
import python_weather

import json
import pandas as pd

# 0. 서비스 함수 준비; 현재 지역의 날씨, 주식 티커 정보, 현재 시각
def get_weather(city:str) -> dict:
    nest_asyncio.apply()
    async def fetch_weekly_weather(city) -> dict:
        client = python_weather.Client(unit=python_weather.METRIC)

        weather = await client.get(city)
        
        retval = {
            "current_weather" : {
                "contry": weather.country,
                "humidity": weather.humidity,
                "feels_like": weather.feels_like,
                "ultraviolet": str(weather.ultraviolet),
                "visibility": weather.visibility,
                "pressure": weather.pressure,
                "kind": str(weather.kind),
                "wind_direction": str(weather.wind_direction),
                "wind_speed": weather.wind_speed,
                "description": weather.description,
                "unit": str(weather.unit),
            },
            "daily_forecast": []
        }

        for forecast in weather.daily_forecasts:
            retval["daily_forecast"].append({
                "date": str(forecast.date),
                "higist_temperature": forecast.highest_temperature,
                "lowest_temperature": forecast.lowest_temperature,
                "sunrise": str(forecast.sunrise),
                "sunset": str(forecast.sunset),
                "unit": str(forecast.unit)
            })

        await client.close()   
        return retval   
    
    return asyncio.run(fetch_weekly_weather(city))

def get_stock_ticker_informations(ticker_symbol:str) -> str:
    return yf.Ticker(ticker_symbol).info

def get_current_time() -> str:
    return str(datetime.datetime.now())

get_stock_ticker_informations("KRW=x")

{'maxAge': 86400,
 'priceHint': 4,
 'previousClose': 1374.43,
 'open': 1374.43,
 'dayLow': 1373.57,
 'dayHigh': 1382.94,
 'regularMarketPreviousClose': 1374.43,
 'regularMarketOpen': 1374.43,
 'regularMarketDayLow': 1373.57,
 'regularMarketDayHigh': 1382.94,
 'bid': 1379.41,
 'ask': 1380.41,
 'fiftyTwoWeekLow': 1216.96,
 'fiftyTwoWeekHigh': 1399.96,
 'fiftyDayAverage': 1340.5436,
 'twoHundredDayAverage': 1324.211,
 'currency': 'KRW',
 'exchange': 'CCY',
 'quoteType': 'CURRENCY',
 'symbol': 'KRW=X',
 'underlyingSymbol': 'KRW=X',
 'shortName': 'USD/KRW',
 'longName': 'USD/KRW',
 'firstTradeDateEpochUtc': 1070236800,
 'timeZoneFullName': 'Europe/London',
 'timeZoneShortName': 'BST',
 'uuid': 'adaa12a3-b8df-3dec-91f1-b942983c3bfd',
 'messageBoardId': 'finmb_KRW_X',
 'gmtOffSetMilliseconds': 3600000,
 'trailingPegRatio': None}

In [28]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# 1. 스레드 생성
thread = client.beta.threads.create()

# 2. 어시스턴트 생성; 함수 호출 도구 추가
assistant = client.beta.assistants.create(
    name="생활 정보 알리미",
    instructions="""
        당신은 나의 생활 정보 알리미입니다. 
        제가 요청하는 정보를 제공한 함수를 적절히 활용해서 알려주세요.
        """,
    model=OPENAI_MODEL,
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "특정 도시의 날씨 정보를 조회합니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "날씨 정보를 조회할 도시의 영문이름, e.g. Seoul"
                        }
                    },
                    "required": ["city"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_stock_ticker_informations",
                "description": "기업의 주식 정보를 조회합니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker_symbol": {
                            "type": "string",
                            "description": "조회할 기업의 티커 심볼, e.g. AAPL"
                        }
                    },
                    "required": ["ticker_symbol"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "현재 시간을 조회 합니다.",
                "parameters": {}
            }
        }
    ]
)

# 3. 사용자 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
        현재 시간과, 서울의 날씨를 알려주세요.
        그리고, 주식 시장의 애플, 마이크로소프트, 삼성전자, 원-달러 환율을 조회해서 알려주세요.
        """
)
  
# 4. 런 생성 및 결과 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 5. 런 상태 확인 및 함수 호출
if "requires_action" == run.status:
    tool_outputs = []
    
    # 어시스턴트가 요청하는 함수를 호출하고, 결과를 ID와 함께 생성
    for tool in run.required_action.submit_tool_outputs.tool_calls:
        function_name = tool.function.name
        args = json.loads(tool.function.arguments)
        
        if "get_weather" == function_name:
            tool_outputs.append({
                "tool_call_id": tool.id,
                "output": json.dumps(get_weather(**args))
            })
        elif "get_stock_ticker_informations" == function_name:
            print(args)
            tool_outputs.append({
                "tool_call_id": tool.id,
                "output": json.dumps(get_stock_ticker_informations(**args))
            })
        elif "get_current_time" == function_name:
            tool_outputs.append({
                "tool_call_id": tool.id,
                "output": json.dumps(get_current_time())
            })
    
    # 어시스턴트에 함수 호출 결과 전달
    if tool_outputs:
        try:
            run = client.beta.threads.runs.submit_tool_outputs_and_poll(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
            print("도구 출력이 성공적으로 제출되었습니다.")
            
        except Exception as e:
            print("도구 결과물을 제출하지 못했습니다.:", e)

# 6. 런 스탭 내용 확인
if "completed" == run.status:
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id,
        run_id=run.id,
        order="asc"
    )
          
    df_run_steps = pd.DataFrame([data.__dict__ for data in run_steps.data])
    
    for details in df_run_steps["step_details"]:
        if "message_creation" == details.type:
            msgid = details.message_creation.message_id
            print(msgid)
            msg = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=msgid
            )
            for content in msg.content:
                print(content.text.value)

else:
    print(run.status)


# 6. 어시스턴트 삭제
res = client.beta.assistants.delete(
    assistant_id=assistant.id
)
print(f"assistant deleted: {res.deleted}")

# 7. 스레드 삭제
res = client.beta.threads.delete(
    thread_id=thread.id
)
print(f"thread deleted: {res.deleted}")


{'ticker_symbol': 'AAPL'}
{'ticker_symbol': 'MSFT'}
{'ticker_symbol': '005930.KS'}
{'ticker_symbol': 'USDKRW=X'}
도구 출력이 성공적으로 제출되었습니다.
msg_S0ndLhqZw8zs1B40FrjTXxhs
**현재 시간 및 서울 날씨 정보:**
- 현재 시간: 2024년 4월 22일 23시 50분 24초
- 서울의 날씨: 맑음
- 현재 온도: 20°C (체감 온도 20°C)
- 최고기온: 22°C, 최저기온: 14°C
- 습도: 46%
- 바람: 동풍 20 km/h

**주식 정보:**
1. **애플 (AAPL)**
   - 현재가: $165.35
   - 오늘 개장가: $165.75
   - 오늘 최고가: $166.23, 최저가: $165.23
   - 시가총액: 약 $2.553조

2. **마이크로소프트 (MSFT)**
   - 현재가: $397.96
   - 오늘 개장가: $400.188
   - 오늘 최고가: $402.5, 최저가: $397.56
   - 시가총액: 약 $2.957조

3. **삼성전자 (005930.KS)**
   - 현재가: 76,100원
   - 오늘 개장가: 77,400원
   - 오늘 최고가: 77,500원, 최저가: 75,100원
   - 시가총액: 약 506.91조 원

4. **원-달러 환율 (USDKRW=X)**
   - 현재 환율: 1,378.30 KRW
   - 오늘 최고: 1,382.94 KRW, 최저: 1,373.57 KRW

이상 요청하신 정보입니다. 추가적인 질문이나 필요한 정보가 있으면 말씀해주세요.
assistant deleted: True
thread deleted: True
